In [ ]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import mne 
from importlib import reload
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
# import own functions
from utils import find_folders
import dat_preproc
import seaborn as sns

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
subID = 'Sub045'

raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'with_med_FTG',
        subID + '_FIF.fif'
    )
)

In [ ]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [ ]:
ps_allamps = []

side = 0


timepoints = [1,	103,	122,	140,	163,	550]
amplitudes = ['preDBS', 'mA_0.5', 'mA_1', 'mA_1.5', 'mA_2', 'postDBS']

for i, tp in enumerate(timepoints):
    ps = np.mean(Sxx[side,:,tp:tp+10],1)
    plt.plot(ps, label = amplitudes[i])
    ps_allamps.append(ps)


plt.xlim(40,100)
plt.ylim(0,0.4)
plt.legend()
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')

In [ ]:
all_ampsDF = pd.DataFrame(np.transpose(ps_allamps))
all_ampsDF.columns = amplitudes
print(all_ampsDF)

In [ ]:
all_ampsDF.to_excel(os.path.join(
    project_path,
   'data', 'power_spectra', 'with_med_FTG',
   subID + '_All_Amps_PS.xlsx'
), index = False
)

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures', 'Power_Spectra','with_med_FTG',
    subID + '_All_Amps_PS'
), dpi = 200
)

In [ ]:
side1 = 0

tp = 103
ps = np.mean(Sxx[side1,:,tp:tp+10],1)
plt.plot(ps, label = amplitudes[i])
plt.xlim(40,100)
plt.ylim(0,0.2)

In [ ]:
peak_freqs = pd.read_excel(os.path.join(
    project_path,
    'results',
    'PeakFreqs_df.xlsx'
))
print(peak_freqs)

In [ ]:
# Reshape the dataframe to have a single column for the boxplot values
df_boxplot = pd.melt(peak_freqs, id_vars=['SubID'], value_vars=['preDBS', 'preSub', 'Sub', 'PostDBS'],
                     var_name='Condition', value_name='Value')

# Plot the boxplots with individual points
grouped_values = df_boxplot.groupby('Condition')['Value'].apply(list)

fig, ax = plt.subplots()

sns.boxplot(x='Condition', y='Value', data=df_boxplot, width=0.5)
sns.stripplot(x='Condition', y='Value', data=df_boxplot, color='black', linewidth=1, jitter=0)

sns.lineplot(
    data=df_boxplot, x='Condition',y='Value', units='SubID',
    color='grey', estimator=None, 
    linestyle=':', lw =  1
)

# Set labels and title
ax.set_xticklabels(['preDBS','Pre-Subharm','Subharm','postDBS'])
plt.ylabel('Peak Frequency [Hz]')
plt.xlabel('')

# Show the plot
plt.show()



In [ ]:
plt.savefig(os.path.join(
    project_path, 'results',
    'PeakFrequencies'
),
dpi = 200)

plt.savefig(os.path.join(
    project_path, 'results',
    'PeakFrequencies.pdf'
))